In [17]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [18]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'f202a70e961d4aa702b6240d6c0b6872', 'Date': 'Fri, 23 Aug 2024 18:52:39 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [20]:
import json 
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Chen is an exceptional instructor. Her lectures are clear, engaging, and she always makes time for student questions. The assignments are challenging but fair.'},
 {'professor': 'Prof. Michael Johnson',
  'subject': 'History',
  'stars': 4,
  'review': "Prof. Johnson brings history to life with his storytelling approach. The workload can be heavy, but it's worth it for the depth of knowledge you gain."},
 {'professor': 'Dr. Sarah Williams',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Dr. Williams is passionate about biology and it shows in every lecture. Her lab sessions are particularly well-organized and informative.'},
 {'professor': 'Prof. David Lee',
  'subject': 'Mathematics',
  'stars': 3,
  'review': "Prof. Lee knows his subject well, but can sometimes struggle to explain complex concepts clearly. Office hours are helpful if you're struggling."},
 {'professor': 'Dr. Rachel Gr

In [27]:
processed_data = []
client = OpenAI()
for review in data["reviews"]:
    print(review["review"])
    response = client.embeddings.create(
        input=review["review"],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }

    })


Dr. Chen is an exceptional instructor. Her lectures are clear, engaging, and she always makes time for student questions. The assignments are challenging but fair.
Prof. Johnson brings history to life with his storytelling approach. The workload can be heavy, but it's worth it for the depth of knowledge you gain.
Dr. Williams is passionate about biology and it shows in every lecture. Her lab sessions are particularly well-organized and informative.
Prof. Lee knows his subject well, but can sometimes struggle to explain complex concepts clearly. Office hours are helpful if you're struggling.
Dr. Green's lectures are fascinating and she often incorporates current research. The class discussions are thought-provoking and engaging.
Prof. Brown has a deep love for literature that's contagious. His essay assignments can be challenging, but they really improve your writing skills.
Dr. Martinez seems knowledgeable, but her teaching style is confusing. The pace of the class is too fast and it's

In [24]:
processed_data[0]

{'values': [-0.0094400365,
  -0.0019483797,
  -0.005036408,
  0.039361466,
  0.05744088,
  -0.0015520854,
  -0.025078729,
  0.027377397,
  -0.025621112,
  0.013365852,
  0.015987366,
  -0.0026441142,
  -0.02364529,
  -0.0012074466,
  0.01823438,
  0.05183626,
  -0.007896829,
  -0.01696882,
  0.0036643096,
  0.042977348,
  0.012694331,
  -0.022663835,
  0.047729652,
  -0.019861527,
  -0.06534416,
  -0.069579914,
  0.026098924,
  -0.023851912,
  0.00019108487,
  0.023929395,
  0.049640905,
  -0.019926097,
  -0.0034576878,
  -0.009427123,
  -0.037346903,
  0.096544065,
  -0.019796958,
  0.029133683,
  0.014114857,
  -0.036675382,
  -0.0049621533,
  0.038354184,
  -0.03830253,
  0.013114031,
  0.012597477,
  -0.034815785,
  -0.01566452,
  0.02274132,
  0.014799291,
  0.007251136,
  -0.057285912,
  0.049821697,
  0.056356113,
  -0.034815785,
  -0.06710045,
  0.033291947,
  0.011945327,
  0.077224925,
  0.0033285492,
  -0.0047910446,
  0.06152166,
  -0.04865945,
  -0.019177092,
  0.011041356

In [25]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [26]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}